<a href="https://colab.research.google.com/github/LBY-L/MineColab/blob/master/MineColab_forked_by_LBY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


```
███╗   ███╗██╗███╗   ██╗███████╗ ██████╗ ██████╗ ██╗      █████╗ ██████╗
████╗ ████║██║████╗  ██║██╔════╝██╔════╝██╔═══██╗██║     ██╔══██╗██╔══██╗
██╔████╔██║██║██╔██╗ ██║█████╗  ██║     ██║   ██║██║     ███████║██████╔╝ 
██║╚██╔╝██║██║██║╚██╗██║██╔══╝  ██║     ██║   ██║██║     ██╔══██║██╔══██╗
██║ ╚═╝ ██║██║██║ ╚████║███████╗╚██████╗╚██████╔╝███████╗██║  ██║██████╔╝
╚═╝     ╚═╝╚═╝╚═╝  ╚═══╝╚══════╝ ╚═════╝ ╚═════╝ ╚══════╝╚═╝  ╚═╝╚═════╝ 
                                  Forked By
                                    LBY_L
```
**Run a Minecraft Server on Google Colab!**

---

This script will make the all of job. Only you need put the server executable in your drive in a new folder named "Minecraft-server" make this when this want the server executable.

In [3]:
import os
import re
import json
from IPython.display import clear_output
# Update the package lists
!sudo apt update &>/dev/null && echo "apt cache successfully updated" || echo "apt cache update failed, you might receive stale packages"

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
# Change directory to the Minecraft server folder on Google Drive
!mkdir "/content/drive/MyDrive/Minecraft-server"
%cd "/content/drive/MyDrive/Minecraft-server"
# Install OpenJDK 17
# !wget -qO - https://adoptopenjdk.jfrog.io/adoptopenjdk/api/gpg/key/public | sudo apt-key add -
# !sudo add-apt-repository --yes https://adoptopenjdk.jfrog.io/adoptopenjdk/deb/ &>/dev/null || echo "Failed to add repo. Still can be ignored if openjdk17 gets installed."
clear_output()
if os.path.isfile("JavaCheck"):
  !if [[ $(cat JavaCheck) == 1 ]]; then sudo apt-get install openjdk-17-jre-headless &>/dev/null && echo "Yay! Openjdk17 has been successfully installed." || echo "Failed to install OpenJdk17." ; elif [[ $(cat JavaCheck) == 2 ]]; then sudo apt-get install openjdk-8-jre-headless &>/dev/null && echo "Yay! Openjdk8 has been successfully installed." || echo "Failed to install OpenJdk8." ; fi
else:
  !touch JavaCheck
  !tput setaf 3; echo "     Select you minecraft version:    "; tput setaf 2; echo "1) 1.17 and above      2) 1.16 to below"; read ver; if [[ $ver == 1 ]]; then echo "1" > JavaCheck; sudo apt-get install openjdk-17-jre-headless &>/dev/null && echo "Yay! Openjdk17 has been successfully installed." || echo "Failed to install OpenJdk17."; elif [[ $ver == 2 ]]; then echo "2" > JavaCheck; sudo apt-get install openjdk-8-jre-headless &>/dev/null && echo "Yay! Openjdk8 has been successfully installed." || echo "Failed to install OpenJdk8."; fi
#Perform java version check
java_ver_17 = !java -version 2>&1 | awk -F[\"\.] -v OFS=. 'NR==1{print $2}'
java_ver_8 = !java -version 2>&1 | awk -F[\"\.] -v OFS=. 'NR==2{print $2}'
if java_ver_17[0] == "17" :
  print("Openjdk17 is working correctly, you are good to go.")
if java_ver_8[0] == "8" :
  print("Openjdk8 is working correctly, you are good to go.")
else:
  print("Openjdk8 or Openjdk17 doesn't seems to be installed or isn't working, falling back to java. You might experience reduced performance. Minecraft 1.17 and above might fail to launch and in 1.16 and below.")

# Import config file.
clear_output()
if os.path.isfile("JarName"):
  JarName = !cat JarName
else:
  !tput setaf 3 ; echo "Which is the name of your jar server name:"
  !read config ; touch JarName ; echo $config > JarName
  !tput setaf 2 ; echo "       You will use PAPER MC (Y/N):       "
  !read -n1 paper ; if [[ $paper == Y ]]; then touch PaperTrue; elif [[ $paper == N ]]; then touch PaperFalse; fi
# Java arguments.
if os.path.isfile("PaperTrue"):
  server_flags = "-XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true"
else:
  server_flags = "" # aiker's flags might negatively impact performance on non-paper servers.
memory_allocation = "-Xmx6144M -Xms6144M"

# Chose the tunnle service you want to use
# Available options: ngrok, playit
clear_output()
if  os.path.isfile("TunnelService"):
  tunnel_service = !cat TunnelService
else:
  !touch "TunnelService"  
  !tput setaf 3 ; echo "What tunnel service will use:"; tput setaf 2 ; echo "1) Ngrok            2) Playit" ; read tunnel; if [[ $tunnel == 1 ]]; then echo "ngrok" > "TunnelService" ; elif [[ $tunnel == 2 ]]; then echo "playit" > "TunnelService"; fi

tunnel_service = !cat TunnelService
JarName = !cat JarName
print("Procedding to use", tunnel_service)
if tunnel_service[0] == "ngrok":
  !pip -q install pyngrok
  from pyngrok import conf, ngrok
  clear_output()
  # Ask for the ngrok authtoken
  !tput setaf 3
  print("Get your authtoken from https://dashboard.ngrok.com/auth")
  !tput setaf 2
  if os.path.isfile("AuthToken"):
    authtoken = !cat AuthToken
  else:
    !touch AuthToken
    !tput setaf 3
    !read -p "Put here your authtoken: " token ; echo "$token" > "AuthToken"
  authtoken = !cat AuthToken
  atk = authtoken[0]
  ! ngrok authtoken $atk # login to ngrok
  Jrnmn = JarName[0]
  print(Jrnmn)
  # Sets default ngrok region
  conf.get_default().region = 'us'  # Change this to whichever region you want

  # Connect to ngrok
  url = ngrok.connect(25565, 'tcp')
  print('Your server address is ' + ((str(url).split('"')[1::2])[0]).replace('tcp://', ''))
  print('Starting server...')
  !java $memory_allocation $server_flags -jar $Jrnmn nogui

elif tunnel_service[0] == "playit":
  clear_output()
  ! curl -SsL https://playit-cloud.github.io/ppa/key.gpg | sudo apt-key add -
  ! sudo curl -SsL -o /etc/apt/sources.list.d/playit-cloud.list https://playit-cloud.github.io/ppa/playit-cloud.list
  ! sudo apt update &>/dev/null && sudo apt install playit &>/dev/null && echo "Playit.gg installed" || echo "Failed to install playit"
  print('Starting server...')
  Jrnmn = JarName[0]
  ! playit & java $memory_allocation $server_flags -jar $Jrnmn nogui


Get your authtoken from https://dashboard.ngrok.com/auth
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
server.jar
Your server address is 2.tcp.ngrok.io:17222
Starting server...
Error: Unable to access jarfile server.jar


# Accept eula

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/Minecraft-server"
!echo "eula=true" > eula.txt

# Questions
---

**Here can do question other time**

In [ ]:
from IPython.display import clear_output
!sudo apt update &>/dev/null && echo "apt cache successfully updated" || echo "apt cache update failed, you might receive stale packages"
from google.colab import drive
drive.mount('/content/drive')
!mkdir "/content/drive/MyDrive/Minecraft-server"
%cd "/content/drive/MyDrive/Minecraft-server"
clear_output()
!touch JavaCheck
!tput setaf 3; echo "     Select you minecraft version:    "; tput setaf 2; echo "1) 1.17 and above      2) 1.16 to below"; read ver; if [[ $ver == 1 ]]; then echo "1" > JavaCheck; sudo apt-get install openjdk-17-jre-headless &>/dev/null && echo "Yay! Openjdk17 has been successfully installed." || echo "Failed to install OpenJdk17."; elif [[ $ver == 2 ]]; then echo "2" > JavaCheck; sudo apt-get install openjdk-8-jre-headless &>/dev/null && echo "Yay! Openjdk8 has been successfully installed." || echo "Failed to install OpenJdk8."; fi
clear_output()
!tput setaf 3 ; echo "Which is the name of your jar server name:"
!read config ; touch JarName ; echo $config > JarName
!rm -r PaperTrue; rm -r PaperFalse
!tput setaf 2 ; echo "       You will use PAPER MC (Y/N):       "
!read -n1 paper ; if [[ $paper == Y ]]; then touch PaperTrue; elif [[ $paper == N ]]; then touch PaperFalse; fi
!touch "TunnelService"
clear_output()
!tput setaf 3 ; echo "What tunnel service will use:"; tput setaf 2 ; echo "1) Ngrok            2) Playit" ; read tunnel; if [[ $tunnel == 1 ]]; then echo "ngrok" > "TunnelService" ; elif [[ $tunnel == 2 ]]; then echo "playit" > "TunnelService"; fi
tunnel_service = !cat TunnelService
print("Procedding to use", tunnel_service)
if tunnel_service[0] == "ngrok":
  clear_output()
  !touch AuthToken
  !tput setaf 3
  !read -p "Put here your authtoken: " token ; echo "$token" > "AuthToken"

# Debug
SSH access to host OS - Thanks to [colab-ssh](https://github.com/WassimBenzarti/colab-ssh).

In [ ]:
#@title Colab-ssh tunnel
#@markdown Execute this cell to open the ssh tunnel. Check [colab-ssh documentation](https://github.com/WassimBenzarti/colab-ssh) for more details.

# Install colab_ssh on google colab
!pip install colab_ssh --upgrade

from colab_ssh import launch_ssh_cloudflared, init_git_cloudflared
ssh_tunnel_password = "<PUT_YOUR_PASSWORD_HERE>" #@param {type: "string"}
launch_ssh_cloudflared(password=ssh_tunnel_password)

In [ ]:
#Get public address (ngrok)
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
## For inspecting the minecraft server directory ##
%cd "/content/drive/My Drive/Minecraft-server"
!ls
